In [3]:
import System.TimeIt

In [4]:
--Exercise 2.1
suffixes' :: [a] -> [[a]]
suffixes' [] = [[]]
suffixes' lst@(x:xs) = lst : suffixes' xs

ex_2_1 :: IO ()
ex_2_1 = timeIt $ print $ foldr seq () $ suffixes' [1..1000000]

ex_2_1

()
CPU time:   0.41s

In [5]:
--Binary Search Tree
data Tree a =
    Empty |
    Tree {
        _left :: Tree a,
        _value :: a,
        _right :: Tree a
    } deriving (Show)
    
member :: Ord a => a -> Tree a -> Bool
member _ Empty = False
member x tree 
    | x < y = member x a
    | y < x = member x b
    | otherwise = True
    where
        a = _left tree
        y = _value tree
        b = _right tree
    
insert :: Ord a => a -> Tree a -> Tree a
insert x Empty = Tree Empty x Empty
insert x tree
    | x < y = Tree (insert x a) y b
    | y < x = Tree a y (insert x b)
    | otherwise = tree
    where
        a = _left tree
        y = _value tree
        b = _right tree

Line 17: Reduce duplication
Found:
a = _left tree
y = _value tree
b = _right tree

Why not:
Combine with -:28:9

In [6]:
--Exercise 2.2
member' :: (Eq a, Ord a) => a -> Tree a -> Bool
member' _ Empty = False
member' x t@(Tree _ y _) = _member y t
    where
        _member z Empty = x == z
        _member z (Tree left val right)
            | x < val = _member z left
            | otherwise = _member val right

In [7]:
--Exercise 2.3
import Data.Maybe

insert' :: Ord a => a -> Tree a -> Tree a
insert' x t = fromMaybe t (_insert t)
    where
        _insert Empty = Just (Tree Empty x Empty)
        _insert (Tree l v r)
            | x < v = fmap (\l -> Tree l v r) (_insert l)
            | x > v = fmap (\r -> Tree l v r) (_insert r)
            | otherwise = Nothing

Line 10: Avoid lambda
Found:
\ r -> Tree l v r
Why not:
Tree l v

In [8]:
--Exercise 2.4
insert :: Ord a => a -> Tree a -> Tree a
insert x t@(Tree _ y _) = fromMaybe t (insert' y t)
    where
        insert' z Empty = if x == z
            then Nothing
            else Just (Tree Empty x Empty)
        insert' z (Tree l v r)
            | x < v = fmap (\l -> Tree l v r) (insert' z l)
            | otherwise = fmap (\r -> Tree l v r) (insert' v r)

Line 10: Avoid lambda
Found:
\ r -> Tree l v r
Why not:
Tree l v

In [9]:
--Exercise 2.5 (a)
complete :: a -> Int -> Tree a
complete x 0 = Empty
complete x d = Tree complete' x complete'
    where complete' = complete x (d-1)

In [10]:
--Exercise 2.5 (b)
balanced :: a -> Int -> Tree a
balanced x 0 = Empty
balanced x 1 = Tree Empty x Empty
balanced x n = if (n-1) `mod` 2 == 0
    then Tree (balanced x ((n-1) `div` 2)) x (balanced x ((n-1) `div` 2))
    else Tree (balanced x ((n-1) `div` 2 + 1)) x (balanced x ((n-1) `div` 2))

In [20]:
--Exercise 2.6
type Map k a = Tree (k, a)

bind :: Ord k => k -> a -> Tree (k, a) -> Tree (k, a)
bind key val Empty = Tree Empty (key, val) Empty
bind key val (Tree left (key_t, val_t) right)
    | key < key_t = Tree (bind key val left) (key_t, val_t) right
    | key > key_t = Tree left (key_t, val_t) (bind key val right)
    | otherwise = Tree left (key_t, val) right

lookup' :: Ord k => k -> Map k a -> a
lookup' key Empty = error "Does not have the key"
lookup' key (Tree left (k_t, v_t) right)
    | key < k_t = lookup' key left
    | key > k_t = lookup' key right
    | otherwise = v_t